In [15]:
#Import relevant packages
import os
import numpy as np
from glob import glob
from shutil import copyfile
from vgg_bn import Vgg16BN
from keras.callbacks import ModelCheckpoint

ROOT_DIR = os.getcwd()
DATA_HOME_DIR = ROOT_DIR
%matplotlib inline

In [29]:
#Change the directory
os.chdir("/home/puneetj/Data_DL")

In [28]:
#importing the Data,applying rescaling and dividing test/validation
data_path = "/home/puneetj/Cats_Dogs" + '/' 
train_path = data_path + 'train/'
valid_path = data_path + 'validation/'
test_path =  data_path + 'test/'
model_path = data_path + 'models/'
submission_path = data_path + 'submissions/'

# data
img_width, img_height = 224, 224
batch_size = 64
nb_train_samples = 20000
nb_valid_samples = 5000
nb_test_samples = 12000
classes = ["cats", "dogs"]
n_classes = len(classes)

# model Parameters
nb_epoch = 10
nb_aug = 5
lr = 0.001

In [30]:
# Importing the pre-fixed model fro bas directory 
vgg = Vgg16BN(size=(img_width, img_height), n_classes=n_classes, batch_size=batch_size, lr=lr)
model = vgg.model

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_3 (Lambda)                (None, 3, 224, 224)   0           lambda_input_3[0][0]             
____________________________________________________________________________________________________
zeropadding2d_27 (ZeroPadding2D) (None, 3, 226, 226)   0           lambda_3[0][0]                   
____________________________________________________________________________________________________
convolution2d_27 (Convolution2D) (None, 64, 224, 224)  1792        zeropadding2d_27[0][0]           
____________________________________________________________________________________________________
zeropadding2d_28 (ZeroPadding2D) (None, 64, 226, 226)  0           convolution2d_27[0][0]           
___________________________________________________________________________________________

/home/puneetj/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_3 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 3, 224, 224)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [34]:
#compiling the model with call backs
vgg.fit(train_path, valid_path,
          nb_trn_samples=nb_train_samples,
          nb_val_samples=nb_valid_samples,
          nb_epoch=nb_epoch,
          callbacks=[ckpt],
          aug=nb_aug)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Epoch 1/10


KeyboardInterrupt: 

In [33]:
info_string = "{0}x{1}_{2}epoch_{3}aug_{4}lr_vgg16-bn".format(img_width, img_height, nb_epoch, nb_aug, lr)
ckpt_fn = model_path + '{val_loss:.2f}-loss_' + info_string + '.h5'

ckpt = ModelCheckpoint(filepath=ckpt_fn,
                      monitor='val_loss',
                      save_best_only=True,
                      save_weights_only=True)

In [147]:
train1 = train[:20000]
train_clas = labels[:20000]
validation = train[20000:]
validation_clas = labels[20000:]

train1 = np.array(train1)
validation = np.array(validation)

In [148]:
n_classes=max(labels)+1
#set shape according to dim ordering
if k.image_dim_ordering() == 'tf':
    train1 = train1.reshape(train1.shape[0],ROWS,COLS,CHANNELS)
    validation = validation.reshape(validation.shape[0],ROWS,COLS,CHANNELS)
    input_shape = (ROWS,COLS,CHANNELS)    
else:
    train1 = train1.reshape(train1.shape[0],CHANNELS,ROWS,COLS)
    validation = validation.reshape(validation.shape[0],CHANNELS,ROWS,COLS)
    input_shape = (CHANNELS,ROWS,COLS)

n_classes=max(labels)+1
train_clas = np_utils.to_categorical(train_clas, nb_classes=n_classes)
validation_clas = np_utils.to_categorical(validation_clas, nb_classes=n_classes)

In [116]:
train1.shape

(20000, 64, 64, 3)

In [153]:
optimizer = RMSprop(lr=.01)
objective = 'binary_crossentropy'


#hyperparameteres
nb_filters1 = 32
nb_filters2 = 64
nb_filters3 = 128


# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (2, 3)

model = Sequential()

model.add(Convolution2D(nb_filters1, kernel_size[0], kernel_size[1],
                        border_mode='same',
                        input_shape=input_shape))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters2, kernel_size[0], kernel_size[1],border_mode='valid'))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size,strides=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(nb_filters3, kernel_size[0], kernel_size[1],
                         border_mode = 'same'))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters3, kernel_size[0], kernel_size[1],border_mode='valid'))
model.add(MaxPooling2D(pool_size=pool_size,strides=(2, 2)))
model.add(Dropout(0.25))

model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size,strides=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(nb_filters3, kernel_size[0], kernel_size[1],
                         border_mode = 'same'))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters3, kernel_size[0], kernel_size[1],border_mode='valid'))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size,strides=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))


model.add(Dense(n_classes))
model.add(BatchNormalization())
model.add(Activation('softmax'))

#compile model


model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=1e-08, decay=0.0),
              metrics=['accuracy'])
              
#run model
batch_size = 64
#using 6 epochs only, for runtime considerations
nb_epoch = 20

train_datagen = ImageDataGenerator(featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)


# fits the model on batches with real-time data augmentation:
model.fit_generator(train_datagen.flow(train1, train_clas, batch_size=64),
                    samples_per_epoch=len(train1), nb_epoch=nb_epoch,
                   verbose =1,validation_data=train_datagen.flow(validation, validation_clas, batch_size=64), nb_val_samples=5000)


#model.fit(train1, train_clas, batch_size=batch_size, nb_epoch=nb_epoch,
         # verbose=1, validation_data=(validation,validation_clas))

Epoch 1/20
   64/20000 [..............................] - ETA: 2038s - loss: 1.0073 - acc: 0.5156

KeyboardInterrupt: 

In [130]:
# saving and scoring models
model.save('model.h5')
score = model.evaluate(train1, train_clas, verbose=1)
pred_cl = model.predict_classes(train)
print('Test score:', score[0])
print('Test accuracy:', score[1])


20000/20000 [==============================] - 670s   


ValueError: Error when checking : expected convolution2d_input_22 to have shape (None, 64, 64, 3) but got array with shape (25000, 3, 64, 64)